## MLflow's Model Registry

In [1]:
# Necessary import
import pandas as pd
import mlflow, pickle
from datetime import datetime
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.metrics import root_mean_squared_error

# Set the mlflow unique resource idenitifier - can be the url of a remote server
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with:
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
# Instantiate an mlflow client
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

# Get the list of ML experiments
client.search_experiments()

[<Experiment: artifact_location='/Users/cm-mboulou-mac/Dvt/MLOps-Zoomcamp-2025/02_experiment-tracking/notebooks/course/mlruns/1', creation_time=1748145819474, experiment_id='1', last_update_time=1748145819474, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748145792148, experiment_id='0', last_update_time=1748145792148, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
# Let's create another experiment
client.create_experiment(name = "my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`:

In [4]:
# Search for runs
runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse < 7",
    run_view_type = ViewType.ACTIVE_ONLY, # to keep only active runs, excluding deleted ones
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

In [5]:
# For each exoeriment run
for run in runs:
    # Print the id and the metrics
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5ed38d6110a1443c8fa388f14adc1c76, rmse: 6.3087
run id: b15f56ed70914fc7980e4d4b547ee569, rmse: 6.7423
run id: a5313a19f9ba49aebd19bc8603e6f28c, rmse: 6.9096
run id: 1edaced899854f70a77a4d24ec146c4b, rmse: 6.9225


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
# Set the tracking server
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
# Experiment run id
run_id = "a5313a19f9ba49aebd19bc8603e6f28c"
# Model URI
model_uri = f"runs:/{run_id}/model"
# Model name
model_name = "nyc-taxi-regressor"
# Register a model
mlflow.register_model(model_uri = model_uri, name = model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748170836107, current_stage='None', description=None, last_updated_timestamp=1748170836107, name='nyc-taxi-regressor', run_id='a5313a19f9ba49aebd19bc8603e6f28c', run_link=None, source='/Users/cm-mboulou-mac/Dvt/MLOps-Zoomcamp-2025/02_experiment-tracking/notebooks/course/mlruns/1/a5313a19f9ba49aebd19bc8603e6f28c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [8]:
# Get the latest versions of the nyc-taxi-regressor model
latest_versions = client.get_latest_versions(name = model_name)

# For each last version of the model
for version in latest_versions:
    # Prints the version and the stage
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None


/var/folders/bc/8szymj2165x0w0nswg6k79xc0000gn/T/ipykernel_27389/4136139139.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


Note that we only have one model here as we couldn't stage models using the interface. This can still be done via code but is deprecated.

In [11]:
# Model version
model_version = 2 # note originally we updated the version 4, and then 1
# Set new stage
new_stage = "Production" # other versions stayed in staging stage
# Transition model from development to staging
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = True # other versions were not archived 
)

/var/folders/bc/8szymj2165x0w0nswg6k79xc0000gn/T/ipykernel_27389/256214788.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748163398233, current_stage='Production', description='', last_updated_timestamp=1748171834425, name='nyc-taxi-regressor', run_id='5ed38d6110a1443c8fa388f14adc1c76', run_link='', source='/Users/cm-mboulou-mac/Dvt/MLOps-Zoomcamp-2025/02_experiment-tracking/notebooks/course/mlruns/1/5ed38d6110a1443c8fa388f14adc1c76/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=2>

In [12]:
# Get today's date
date = datetime.today().date()
# Update the model's decription by its version (here with the model v2)
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"The model version {model_version} was transitioned to {new_stage} on {date}."
)

<ModelVersion: aliases=[], creation_timestamp=1748163398233, current_stage='Production', description='The model version 2 was transitioned to Production on 2025-05-25.', last_updated_timestamp=1748172163949, name='nyc-taxi-regressor', run_id='5ed38d6110a1443c8fa388f14adc1c76', run_link='', source='/Users/cm-mboulou-mac/Dvt/MLOps-Zoomcamp-2025/02_experiment-tracking/notebooks/course/mlruns/1/5ed38d6110a1443c8fa388f14adc1c76/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [13]:
# Function for wrangling the data
def read_dataframe(filename):
    # If the data is a csv file
    if filename.endswith('.csv'):
        # Read csv
        df = pd.read_csv(filename)
        # Convert to datetime
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    # else if it is a parquet file
    elif filename.endswith('.parquet'):
        # read parquet
        df = pd.read_parquet(filename)
    # Not considered file type
    else:
        print("File not supported") # error message

    # Feature Engineering
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime # target var
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # for minutes
    # Filtering durations
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical features selection
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str) # convert integers to string

    # return the dataframe
    return df

# Function to preprocess the data
def preprocess(df, dv):
    # Get the trajet column
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    # Features selection
    categorical = ['PU_DO'] # categorical
    numerical = ['trip_distance'] # numerical
    # Train data dictionaries
    train_dicts = df[categorical + numerical].to_dict(orient = 'records')
    # return preprocessed data
    return dv.transform(train_dicts)

# Function to test the model
def test_model(name, stage, X_test, y_test):
    # Load the model
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    # Make a prediction
    y_pred = model.predict(X_test)
    # Return the RMSE
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

Note that the [March dataset](curl -O "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet") has been downloaded using the terminal.

In [15]:
# Read the data
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [16]:
# Download the encoder - we used one preprocessor so no need to check for the right id
client.download_artifacts(run_id = run_id, path ='preprocessor', dst_path = '.')

'/Users/cm-mboulou-mac/Dvt/MLOps-Zoomcamp-2025/02_experiment-tracking/notebooks/course/preprocessor'

In [17]:
# Open the encoder file
with open("preprocessor/preprocessor.b", "rb") as f_in:
    # Get the encoder model
    dv = pickle.load(f_in)

In [18]:
# Preprocess the testing data
X_test = preprocess(df, dv)

# Target variable
target = "duration"
# Real target values
y_test = df[target].values

In [21]:
%time test_model(name = model_name, stage = "Production", X_test = X_test, y_test = y_test) # test model in production

CPU times: user 24.4 s, sys: 205 ms, total: 24.6 s
Wall time: 3.58 s


{'rmse': 6.256345322997512}

In [22]:
%time test_model(name = model_name, stage = "Staging", X_test = X_test, y_test = y_test) # test model in staging

CPU times: user 5.82 s, sys: 247 ms, total: 6.07 s
Wall time: 6.27 s


{'rmse': 6.885693344710411}

In [23]:
# Model can be transitioned as Production as followed:
# client.transition_model_version_stage(
#     name = model_name,
#     version = 4,
#     stage = "Production",
#     archive_existing_versions = True
# )
# If the performaned was similar to the optimal we would have done that.

---